# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import nltk
import re
import os
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

nltk.download(['punkt', 'wordnet']);

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///ETL_Data.db')
df = pd.read_sql('ETL_Data', engine)


# seperate dataset to input variables (X) and target variable (Y)
X = df.message
Y = df[df.columns[4:]]

df


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Takes a text string as input and performs several text preprocessing steps on it including:
    - replacing URLs in the text with a placeholder
    - tokenizing the text into individual words
    - lemmatizing the words to their base form
    - converting the words to lowercase
 
    Arguments:
    text -- the input text string to be processed

    Returns:
    clean_tokens -- a list of cleaned tokens extracted from the input text string
    """

    # replace all urls wirth a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # extract urls from given text
    detected_urls = re.findall(url_regex, text)

    # replace all urls to 'url'
    for url in detected_urls:
        text = text.replace(url, "url")

    # extract the word tokens
    tokens = word_tokenize(text)

    # lemmatize to the stem of a word
    lemmatizer = WordNetLemmatizer()
    
    # list clean tokens
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# define the pipeline
# this pipeline can be trained on a dataset of labeled messages and used to predict the category of new, unlabeled messages based on the trained model.
pipeline = Pipeline([
        # counts the number of occurrences of each word in a text message
        # it takes an optional tokenizer parameter, which in this case is a custom tokenize function that splits a text message into individual words.
        ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
        # computes the TF-IDF (term frequency-inverse document frequency) scores for the words in the text messages
        # this is a way of weighing the importance of each word based on how frequently it appears in the messages 
        # and how common it is across all messages in the dataset.
        ('tfidf', TfidfTransformer()),
        # type of machine learning classifier that uses a collection of decision trees to classify messages into different categories
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)


def class_report(y_test, y_pred):
   """
   Prints the classification report containing precision, recall, f1 score and support 
   for each label in y_test and y_pred.
   
   Args:
   y_test (pandas.DataFrame): DataFrame with true labels.
   y_pred (numpy.ndarray): NumPy array with predictions generated by a trained model.
   
   Returns:
   None
   """
   # Loop through each column of y_test
   for i, col in enumerate(y_test.columns):
       print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
       # Print the name of the column
       print(col, ':')
       print('-----------------------------------------------------')
       # Print the classification report for y_test and y_pred at ith column
       print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

# Call the class_report function with y_test and y_pred to print the classification report
class_report(y_test, y_pred)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
related :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.65      0.33      0.44      1535
          1       0.82      0.94      0.88      4974
          2       0.82      0.20      0.32        45

avg / total       0.78      0.80      0.77      6554

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
request :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5421
          1       0.85      0.36      0.51      1133

avg / total       0.88      0.88      0.86      6554

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
offer :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# display pipelines parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern=None,
           tokenizer=<function tokenize at 0x7f0571116950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
   

In [10]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
# Get the prediction values from the grid search cross validator
y_pred_test_cv = cv.predict(X_test)
y_pred_train_cv = cv.predict(X_train)

In [13]:
# Print the best parameters
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__min_samples_split': 2, 'tfidf__norm': 'l2'}


In [14]:
class_report(y_test, y_pred_test_cv)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
related :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.63      0.36      0.46      1535
          1       0.82      0.93      0.87      4974
          2       0.69      0.20      0.31        45

avg / total       0.77      0.79      0.77      6554

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
request :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5421
          1       0.82      0.34      0.48      1133

avg / total       0.87      0.87      0.85      6554

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
offer :
-----------------------------------------------------
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6520
          1       0.00      0.00      0.00        34

avg 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Improve the model by adding the StartingVerbExtractor
from sklearn.pipeline import FeatureUnion
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

# Build the new Pipeline
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

# New Parameters for the grid search
parameters2 = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]
    }

# Create new grid search
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


### 9. Export your model as a pickle file

In [16]:
# Save the old trained model to a file
with open('classifier_old.pkl', 'wb') as file:
    pickle.dump(cv, file)

# Save the improved trained model to a file
with open('classifier.pkl', 'wb') as file:
    pickle.dump(cv2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.